In [1]:
from datacube import Datacube
import pandas as pd
import json
import re

import sys
import pystac_client
import odc.stac
import timeit
import pstats
import cProfile
sys.path.insert(1, "../Tools/")
from dea_tools.plotting import rgb
odc.stac.configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)
stac_prod = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac")
stac_dev = pystac_client.Client.open("https://explorer.dev.dea.ga.gov.au/stac")

/home/ubuntu/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#dc_dev = Datacube(app='', env='dev')
#dc_prod = Datacube(app='')
time_range = ['2024-04-01', '2025-07-01']
time_to_drill = ("2025-01", "2025-03")

# Set a start and end date
start_date = "2025-03-01"
end_date = "2025-03-31"

# Set product ID as the STAC "collection"
collections = ["ga_ls9c_ard_3"]

## ARD

In [ ]:
query = {
    'time': ('2025-01-01', '2025-09-06'),
    'product':'ga_ls9c_ard_3',
}
# dev_ards=dc_dev.find_datasets(**query)
#prod_ards=dc_prod.find_datasets(**query)
# ids_dev = [str(dsdev.id) for dsdev in dev_ards]
#ids_prod = [str(dsprod.id) for dsprod in prod_ards]
# print(len(ids_dev))
#print(len(ids_prod))
# print(ids_dev[:10])
#print(ids_prod[:10])
#dev_ard_ls8 = dc_dev.find_datasets(product='ga_ls8c_ard_3', time=time_range)
#dev_ard_ls9 = dc_dev.find_datasets(product='ga_ls9c_ard_3', time=time_range)
#dev_ard_all = dev_ard_ls8 #+ dev_ard_ls9
# dev_ards = stac_prod.search(
#     collections=collections,
#     datetime=f"{start_date}/{end_date}",    
# )
stac_prod = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac")


ga_ls9c_ard_3_aug25_catalog = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac/catalogs/ga_ls9c_ard_3/2025-8?limit=999")
print(len(ga_ls9c_ard_3_aug25_catalog.get_links()))
ga_ls9c_ard_3_aug25_datasets = list(ga_ls9c_ard_3_aug25_catalog.get_links())
print(ga_ls9c_ard_3_aug25_datasets[3].get_target_str())
print(ga_ls9c_ard_3_lnks[3].title)
print(len(ga_ls9c_ard_3_aug25_datasets))
print(ga_ls9c_ard_3_aug25_catalog.extra_fields)

start_date = "2025-03-01"
end_date = "2025-03-31"
collections = ["ga_ls9c_ard_3"]

# collections = list(stac_prod.get_collections())
# print(f"Number of collections: {len(collections)}")
# print("Collections IDs:")
# for collection in collections:
#     print(f"- {collection.id}")

# execution_time = timeit.timeit(prod_stac_function, number=1)
# print(f"Execution time: {execution_time:.6f} seconds")
collection = stac_prod.get_child("ga_ls9c_ard_3")
ard_collection = stac_prod.get_collection("ga_ls9c_ard_3")
print(f"Collection ID: {collection.id}")
print(f"Title: {collection.title}")
print(f"Description: {collection.description}")
# print(collection.get_item_links())
# for i in stac_prod.get_items(recursive=True):
#     print(i.id)
def prod_stac_function():
    prod_ards = stac_prod.search(
        collections=collections,
        datetime=f"{start_date}/{end_date}", 
        fields={"include": ["ids"]},
        #max_items=1,
        limit=100,    
    )
    items_prod=list(prod_ards.items())
    ids_prod = [str(dsprod.id) for dsprod in items_prod]
    print(prod_ards.matched())
    # print(ids_dev[:10])
    print(ids_prod[:1])
    # for item in prod_ards.items():
    #     print(item.id)

# execution_time = timeit.timeit(prod_stac_function(), number=1)
# print(f"Execution time: {execution_time:.6f} seconds")

cProfile.runctx("prod_stac_function()",None, locals(),'my_func_stats')

p = pstats.Stats("my_func_stats")
p.sort_stats("cumulative").print_stats()
# Search the STAC catalog for all items matching the query
#ids_dev=list(dev_ards)      
#ids_prod=list(prod_ards)
#prod_ard_ls8 = dc_prod.find_datasets(product='ga_ls8c_ard_3', time=time_range)
#prod_ard_ls9 = dc_prod.find_datasets(product='ga_ls9c_ard_3', time=time_range)
#prod_ard_all = prod_ard_ls8 #+ prod_ard_ls9
# dev_items = list(dev_ards.items())
# prod_items = list(prod_ards.items())
# print(f"Found: {len(dev_items):d} datasets")

869
https://explorer.dea.ga.gov.au/stac/collections/ga_ls9c_ard_3/items/4eb8a67a-877f-4a0f-aa61-f239636f3f1f
<built-in method title of str object at 0x7560abdc68d0>
869
{'type': 'Catalog', 'numberReturned': 867, 'numberMatched': 861}


/home/ubuntu/.local/lib/python3.12/site-packages/pystac_client/client.py:191: NoConformsTo: Server does not advertise any conformance classes.
  warnings.warn(NoConformsTo())


In [ ]:
# get only final maturity
final_prod_ard = [
    ds
    for ds in prod_ard_all
    if ds.metadata_doc.get("properties", {}).get("dea:dataset_maturity") == "final"
]

# count unique IDs by path and row
ard_path_row_unique_ids = {}

for ds in final_prod_ard:
    props = ds.metadata_doc.get("properties", {})
    path = props.get("landsat:wrs_path")
    row = props.get("landsat:wrs_row")
    unique_id = ds.metadata_doc.get("id")  # Use the UUID as unique scene identifier
    if path is None or row is None or unique_id is None:
        continue
    key = (path, row)
    if key not in ard_path_row_unique_ids:
        ard_path_row_unique_ids[key] = set()
    ard_path_row_unique_ids[key].add(unique_id)
ard_path_row_unique_ids
# Now build DataFrame with counts of unique scene IDs
df = pd.DataFrame(
    [{"path": k[0], "row": k[1], "count": len(v)} for k, v in ard_path_row_unique_ids.items()]
)
pivot_df = df.pivot(index="row", columns="path", values="count")
# mirror columns to match order of scenes
pivot_df = pivot_df[pivot_df.columns[::-1]]

pivot_df.index.name = ""
pivot_df.columns.name = ""

pivot_df.to_csv("ARD_prod_ls8ls9_count_final_uniqueid.csv")


NameError: name 'prod_ard_all' is not defined

In [ ]:
# get only final maturity
final_dev_ard = [
    ds
    for ds in dev_ard_all
    if ds.metadata_doc.get("properties", {}).get("dea:dataset_maturity") == "final"
]

# count unique IDs by path and row
ard_path_row_unique_ids = {}

for ds in final_dev_ard:
    props = ds.metadata_doc.get("properties", {})
    path = props.get("landsat:wrs_path")
    row = props.get("landsat:wrs_row")
    unique_id = ds.metadata_doc.get("id")  # Use the UUID as unique scene identifier
    if path is None or row is None or unique_id is None:
        continue
    key = (path, row)
    if key not in ard_path_row_unique_ids:
        ard_path_row_unique_ids[key] = set()
    ard_path_row_unique_ids[key].add(unique_id)

# Now build DataFrame with counts of unique scene IDs
df = pd.DataFrame(
    [{"path": k[0], "row": k[1], "count": len(v)} for k, v in ard_path_row_unique_ids.items()]
)
pivot_df = df.pivot(index="row", columns="path", values="count")
# mirror columns to match order of scenes
pivot_df = pivot_df[pivot_df.columns[::-1]]

pivot_df.index.name = ""
pivot_df.columns.name = ""

pivot_df.to_csv("ARD_dev_ls8ls9_count_final_uniqueid.csv")


In [ ]:
prod_count = pd.read_csv('ARD_prod_ls8ls9_count_final_uniqueid.csv', index_col=0)
dev_count = pd.read_csv('ARD_dev_ls8ls9_count_final_uniqueid.csv', index_col=0)

# reduce to columns in common
prod_count = prod_count[[c for c in prod_count.columns if c in dev_count.columns]]
dev_count = dev_count[[c for c in prod_count.columns]]

# reduce to same index 
common_index = prod_count.index.intersection(dev_count.index)
prod_count = prod_count.loc[common_index]
dev_count = dev_count.loc[common_index]

# double check
assert prod_count.columns.equals(dev_count.columns)
assert prod_count.index.equals(dev_count.index)

In [ ]:
diff = prod_count - dev_count
diff.to_csv('diff_dev_prod_count_unique_final.csv')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# use the existing diff DataFrame as starting point
diff_df = pd.read_csv('diff_dev_prod_count_unique_final.csv', index_col=0)

# heatmap without cell annotations
plt.figure(figsize=(14, 10))
sns.set(style="whitegrid")

ax = sns.heatmap(
    diff_df,
    fmt='',
    cmap='YlOrRd',
    linewidths=0.5,
    linecolor='gray',
    square=True,
    cbar_kws={'label': 'Missing Scenes Count'},
    vmin=0,
    # vmax=2
)

ax.set_title('LS8 + LS9 ARD: Prod - Dev', fontsize=14)
ax.set_xlabel('Path')
ax.set_ylabel('Row')
plt.tight_layout()
plt.show()


In [ ]:
dev_wofs = dc_dev.find_datasets(product='ga_ls_wo_3', time=time_range)

dev_wofs = dc_dev.find_datasets(product='ga_ls_wo_3', time=time_range)

prod_wofs = dc_prod.find_datasets(product='ga_ls_wo_3', time=time_range)
c = list(set(prod_wofs) - set(dev_wofs))

print(c)

In [ ]:
# Filter for final maturity only
final_dev_wofs = [
    ds
    for ds in dev_wofs
    if ds.metadata_doc.get("properties", {}).get("dea:dataset_maturity") == "final"
]

# Count unique scene IDs by path and row
wofs_path_row_unique_ids = {}

for ds in final_dev_wofs:
    props = ds.metadata_doc.get("properties", {})
    path = props.get("landsat:wrs_path")
    row = props.get("landsat:wrs_row")
    unique_id = ds.metadata_doc.get("id")  # UUID as the unique identifier
    if path is None or row is None or unique_id is None:
        continue
    key = (path, row)
    if key not in wofs_path_row_unique_ids:
        wofs_path_row_unique_ids[key] = set()
    wofs_path_row_unique_ids[key].add(unique_id)
dev_wofs_path_row_unique_ids=wofs_path_row_unique_ids
# Create DataFrame with counts
df = pd.DataFrame(
    [{"path": k[0], "row": k[1], "count": len(v)} for k, v in wofs_path_row_unique_ids.items()]
)

pivot_df = df.pivot(index="row", columns="path", values="count")

# Reverse columns (if needed)
pivot_df = pivot_df[pivot_df.columns[::-1]]

# Clean up axis labels
pivot_df.index.name = ""
pivot_df.columns.name = ""

# Export to CSV
pivot_df.to_csv("wofs_dev_count_final_uniqueid.csv")

In [ ]:
# Filter for final maturity only
final_prod_wofs = [
    ds
    for ds in prod_wofs
    if ds.metadata_doc.get("properties", {}).get("dea:dataset_maturity") == "final"
]

# Count unique scene IDs by path and row
wofs_path_row_unique_ids = {}

for ds in final_prod_wofs:
    props = ds.metadata_doc.get("properties", {})
    path = props.get("landsat:wrs_path")
    row = props.get("landsat:wrs_row")
    unique_id = ds.metadata_doc.get("id")  # UUID as the unique identifier
    if path is None or row is None or unique_id is None:
        continue
    key = (path, row)
    if key not in wofs_path_row_unique_ids:
        wofs_path_row_unique_ids[key] = set()
    wofs_path_row_unique_ids[key].add(unique_id)
prod_wofs_path_row_unique_ids=wofs_path_row_unique_ids
# Create DataFrame with counts
df = pd.DataFrame(
    [{"path": k[0], "row": k[1], "count": len(v)} for k, v in wofs_path_row_unique_ids.items()]
)

pivot_df = df.pivot(index="row", columns="path", values="count")

# Reverse columns (if needed)
pivot_df = pivot_df[pivot_df.columns[::-1]]

# Clean up axis labels
pivot_df.index.name = ""
pivot_df.columns.name = ""

# Export to CSV
pivot_df.to_csv("wofs_prod_count_final_uniqueid.csv")

In [ ]:
prod_count = pd.read_csv('wofs_prod_count_final_uniqueid.csv', index_col=0)
dev_count = pd.read_csv('wofs_dev_count_final_uniqueid.csv', index_col=0)

# reduce to columns in common
prod_count = prod_count[[c for c in prod_count.columns if c in dev_count.columns]]
dev_count = dev_count[[c for c in prod_count.columns]]

# reduce to same index 
common_index = prod_count.index.intersection(dev_count.index)
prod_count = prod_count.loc[common_index]
dev_count = dev_count.loc[common_index]

# double check
assert prod_count.columns.equals(dev_count.columns)
assert prod_count.index.equals(dev_count.index)


In [ ]:
diff = prod_count - dev_count
diff.to_csv('wo_diff_dev_prod_count_final_uniqueid.csv')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# use the existing diff DataFrame as starting point
diff_df = pd.read_csv('wo_diff_dev_prod_count_final_uniqueid.csv', index_col=0)

# heatmap without cell annotations
plt.figure(figsize=(14, 10))
sns.set(style="whitegrid")

ax = sns.heatmap(
    diff_df,
    fmt='',
    cmap='YlOrRd',
    linewidths=0.5,
    linecolor='gray',
    square=True,
    cbar_kws={'label': 'Missing Scenes Count'},
    vmin=0,
    # vmax=2
)

ax.set_title('WO: Prod - Dev', fontsize=14)
ax.set_xlabel('Path')
ax.set_ylabel('Row')
plt.tight_layout()
plt.show()


In [ ]:
dev_fc = dc_dev.find_datasets(product='ga_ls_fc_3', time=time_range)
prod_fc = dc_prod.find_datasets(product='ga_ls_fc_3', time=time_range)
c = list(set(prod_fc) - set(dev_fc))

print(c)

In [ ]:
# Filter for final maturity only
final_dev_fc = [
    ds
    for ds in dev_fc
    if ds.metadata_doc.get("properties", {}).get("dea:dataset_maturity") == "final"
]

# Count unique scene IDs by path and row
fc_path_row_unique_ids = {}

for ds in final_dev_fc:
    props = ds.metadata_doc.get("properties", {})
    path = props.get("landsat:wrs_path")
    row = props.get("landsat:wrs_row")
    unique_id = ds.metadata_doc.get("id")  # UUID as the unique identifier
    if path is None or row is None or unique_id is None:
        continue
    key = (path, row)
    if key not in fc_path_row_unique_ids:
        fc_path_row_unique_ids[key] = set()
    fc_path_row_unique_ids[key].add(unique_id)
# Create DataFrame with counts
df = pd.DataFrame(
    [{"path": k[0], "row": k[1], "count": len(v)} for k, v in fc_path_row_unique_ids.items()]
)

pivot_df = df.pivot(index="row", columns="path", values="count")

# Reverse columns (if needed)
pivot_df = pivot_df[pivot_df.columns[::-1]]

# Clean up axis labels
pivot_df.index.name = ""
pivot_df.columns.name = ""

# Export to CSV
pivot_df.to_csv("fc_dev_count_final_uniqueid.csv")

In [ ]:
# Filter for final maturity only
final_prod_fc = [
    ds
    for ds in prod_fc
    if ds.metadata_doc.get("properties", {}).get("dea:dataset_maturity") == "final"
]

# Count unique scene IDs by path and row
fc_path_row_unique_ids = {}

for ds in final_prod_fc:
    props = ds.metadata_doc.get("properties", {})
    path = props.get("landsat:wrs_path")
    row = props.get("landsat:wrs_row")
    unique_id = ds.metadata_doc.get("id")  # UUID as the unique identifier
    if path is None or row is None or unique_id is None:
        continue
    key = (path, row)
    if key not in fc_path_row_unique_ids:
        fc_path_row_unique_ids[key] = set()
    fc_path_row_unique_ids[key].add(unique_id)
fc_path_row_unique_ids
# Create DataFrame with counts
df = pd.DataFrame(
    [{"path": k[0], "row": k[1], "count": len(v)} for k, v in fc_path_row_unique_ids.items()]
)

pivot_df = df.pivot(index="row", columns="path", values="count")

# Reverse columns (if needed)
pivot_df = pivot_df[pivot_df.columns[::-1]]

# Clean up axis labels
pivot_df.index.name = ""
pivot_df.columns.name = ""

# Export to CSV
pivot_df.to_csv("fc_prod_count_final_uniqueid.csv")

In [ ]:
prod_count = pd.read_csv('fc_prod_count_final_uniqueid.csv', index_col=0)
dev_count = pd.read_csv('fc_dev_count_final_uniqueid.csv', index_col=0)

# reduce to columns in common
prod_count = prod_count[[c for c in prod_count.columns if c in dev_count.columns]]
dev_count = dev_count[[c for c in prod_count.columns]]

# reduce to same index 
common_index = prod_count.index.intersection(dev_count.index)
prod_count = prod_count.loc[common_index]
dev_count = dev_count.loc[common_index]

# double check
assert prod_count.columns.equals(dev_count.columns)
assert prod_count.index.equals(dev_count.index)


In [ ]:
diff = prod_count - dev_count
diff.to_csv('fc_diff_dev_prod_count_final_uniqueid.csv')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# use the existing diff DataFrame as starting point
diff_df = pd.read_csv('fc_diff_dev_prod_count_final_uniqueid.csv', index_col=0)

# heatmap without cell annotations
plt.figure(figsize=(14, 10))
sns.set(style="whitegrid")

ax = sns.heatmap(
    diff_df,
    fmt='',
    cmap='YlOrRd',
    linewidths=0.5,
    linecolor='gray',
    square=True,
    cbar_kws={'label': 'Missing Scenes Count'},
    vmin=0,
    # vmax=2
)

ax.set_title('FC: Prod - Dev', fontsize=14)
ax.set_xlabel('Path')
ax.set_ylabel('Row')
plt.tight_layout()
plt.show()


In [ ]:
def extract_final_titles(dataset_list):
    titles = set()
    for ds in dataset_list:
        props = ds.metadata_doc.get("properties", {})
        if props.get("dea:dataset_maturity") != "final":
            continue
        title = props.get("title")
        if title:
            titles.add(title)
    return titles

dev_titles = extract_final_titles(dev_fc)
prod_titles = extract_final_titles(prod_fc)

missing_in_dev = sorted(prod_titles - dev_titles)
missing_in_prod = sorted(dev_titles - prod_titles)


In [ ]:
missing_in_dev

In [ ]:
import pandas as pd
from collections import defaultdict

def find_missing_ids_all_path_rows(dev_fc, prod_fc):
    def extract_ids_by_path_row(dataset_list):
        ids_dict = defaultdict(set)
        for ds in dataset_list:
            props = ds.metadata_doc.get("properties", {})
            if props.get("dea:dataset_maturity") != "final":
                continue
            path = props.get("landsat:wrs_path")
            row = props.get("landsat:wrs_row")
            if path is None or row is None:
                continue
            ids_dict[(path, row)].add(ds.id)
        return ids_dict

    dev_ids_by_path_row = extract_ids_by_path_row(dev_fc)
    prod_ids_by_path_row = extract_ids_by_path_row(prod_fc)

    all_keys = set(dev_ids_by_path_row.keys()) | set(prod_ids_by_path_row.keys())

    missing_in_dev = {}
    missing_in_prod = {}

    for key in all_keys:
        dev_ids = dev_ids_by_path_row.get(key, set())
        prod_ids = prod_ids_by_path_row.get(key, set())

        missing_dev = sorted(prod_ids - dev_ids)
        missing_prod = sorted(dev_ids - prod_ids)

        if missing_dev:
            missing_in_dev[key] = missing_dev
        if missing_prod:
            missing_in_prod[key] = missing_prod

    # Print summary
    for (path, row), missing_ids in missing_in_dev.items():
        print(f"Missing in DEV for path={path}, row={row}:")
        for _id in missing_ids:
            print(f"  {_id}")
        print()

    for (path, row), missing_ids in missing_in_prod.items():
        print(f"Missing in PROD for path={path}, row={row}:")
        for _id in missing_ids:
            print(f"  {_id}")
        print()

    return missing_in_dev, missing_in_prod


In [ ]:
missing_dev, missing_prod = find_missing_ids_all_path_rows(dev_fc, prod_fc)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def find_missing_scenes_grouped(dev_fc, prod_fc):
    # Helper to extract scene IDs grouped by path and row
    def scenes_by_path_row(datasets):
        d = {}
        for ds in datasets:
            props = ds.metadata_doc.get("properties", {})
            if props.get("dea:dataset_maturity") != "final":
                continue
            p = props.get("landsat:wrs_path")
            r = props.get("landsat:wrs_row")
            sid = props.get("landsat:landsat_scene_id") or props.get("title")
            if p is None or r is None or sid is None:
                continue
            d.setdefault((p, r), set()).add(sid)
        return d

    dev_scenes = scenes_by_path_row(dev_fc)
    prod_scenes = scenes_by_path_row(prod_fc)

    missing_in_dev = {}
    missing_in_prod = {}

    all_keys = set(dev_scenes.keys()).union(prod_scenes.keys())

    for key in all_keys:
        dev_set = dev_scenes.get(key, set())
        prod_set = prod_scenes.get(key, set())

        missing_dev = prod_set - dev_set
        missing_prod = dev_set - prod_set

        if missing_dev:
            missing_in_dev[key] = list(missing_dev)
        if missing_prod:
            missing_in_prod[key] = list(missing_prod)

    return missing_in_dev, missing_in_prod

def prepare_heatmap_data(missing_in_dev, dev_fc, prod_fc):
    # Collect all path/row combos from dev and prod datasets
    def get_all_path_rows(datasets):
        combos = set()
        for ds in datasets:
            props = ds.metadata_doc.get("properties", {})
            if props.get("dea:dataset_maturity") != "final":
                continue
            p = props.get("landsat:wrs_path")
            r = props.get("landsat:wrs_row")
            if p is not None and r is not None:
                combos.add((p, r))
        return combos

    dev_combos = get_all_path_rows(dev_fc)
    prod_combos = get_all_path_rows(prod_fc)

    all_combos = dev_combos.union(prod_combos)

    # Extract unique paths and rows (sorted)
    all_paths = sorted({p for p, r in all_combos})
    all_rows = sorted({r for p, r in all_combos})

    # Initialize DataFrame with NaNs for all combos
    df_full = pd.DataFrame(np.nan, index=all_rows, columns=all_paths)

    # Fill in zero for combos with data but no missing scenes
    for (path, row) in all_combos:
        df_full.at[row, path] = 0

    # Overwrite with missing counts where applicable
    for (path, row), missing_scenes in missing_in_dev.items():
        df_full.at[row, path] = len(missing_scenes)

    return df_full

# Run missing scene detection
missing_in_dev, missing_in_prod = find_missing_scenes_grouped(dev_fc, prod_fc)

# Prepare full heatmap data including zero counts
heatmap_data = prepare_heatmap_data(missing_in_dev, dev_fc, prod_fc)

# Plot heatmap
plt.figure(figsize=(14, 10))
sns.set(style="whitegrid")

ax = sns.heatmap(
    heatmap_data,
    cmap='YlOrRd',
    linewidths=0.5,
    linecolor='gray',
    square=True,
    cbar_kws={'label': 'Missing Scenes Count'},
    vmin=0
)

ax.set_title('Count of Missing Scenes in DEV by Path and Row', fontsize=14)
ax.set_xlabel('Path')
ax.set_ylabel('Row')
plt.tight_layout()
plt.show()


In [ ]:
def find_missing_ids_by_path_row(dev_fc, prod_fc, target_path, target_row):
    def extract_ids(datasets, path, row):
        ids = set()
        for ds in datasets:
            props = ds.metadata_doc.get("properties", {})
            if props.get("dea:dataset_maturity") != "final":
                continue
            try:
                p = int(props.get("landsat:wrs_path"))
                r = int(props.get("landsat:wrs_row"))
            except (TypeError, ValueError):
                continue
            if p == path and r == row:
                ids.add(ds.id)
        return ids

    dev_ids = extract_ids(dev_fc, target_path, target_row)
    prod_ids = extract_ids(prod_fc, target_path, target_row)

    missing_in_dev = sorted(prod_ids - dev_ids)
    missing_in_prod = sorted(dev_ids - prod_ids)

    print(f"Missing IDs in DEV (present in PROD) for path={target_path}, row={target_row}:")
    for mid in missing_in_dev:
        print(f"  {mid}")

    print(f"\nMissing IDs in PROD (present in DEV) for path={target_path}, row={target_row}:")
    for mid in missing_in_prod:
        print(f"  {mid}")

    return missing_in_dev, missing_in_prod


In [ ]:
missing_in_dev, missing_in_prod = find_missing_ids_by_path_row(dev_fc, prod_fc,106, 78) #path, row
missing_in_prod
missing_in_dev

In [ ]:
missing_in_dev, missing_in_prod = find_missing_ids_by_path_row(dev_wofs, prod_wofs, 99, 85) #path, row


# Example read metadata

In [ ]:
#ds = wofs[10]
#print(json.dumps(ds.metadata_doc, indent=2))


In [ ]:
# ds = ard_all[10]
# print(json.dumps(ds.metadata_doc, indent=2))